# Exercise 9: Multinomial Regression

In multinomial regression, we have $K$ classes and inputs with $n$ features, i.e., $\mathcal{X} = \mathbb{R}^n$ and $\mathcal{Y} = \{ 1, \ldots, K \}$.

The hypothesis, for class $k$, is the estimated probability $\phi_k$ that $\mathbf{x}$ was drawn from class $k$ using
the linear transformation of the input followed by the softmax function:

$$\phi_k = \frac{e^{\theta_k^T \mathbf{x}}}{\sum_{j=1}^K e^{\theta_j^T \mathbf{x}}}.$$

The cost function is the *cross entropy* loss function
$$J(\theta) = -\sum_{i=1}^m \log \phi_{y^{(i)}}.$$

The algorithm to minimize the cost $J(\theta)$ is just gradient descent. The batch version of the gradient is
$$ \frac{\partial J}{\partial \theta_{kl}} = - \sum_{i=1}^m (\delta(y^{(i)} = k) - \phi_k)x_l^{(i)}.$$

To demonstrate, we begin by reading the famous "Iris" dataset from [The UCI ML Dataset Repository](https://archive.ics.uci.edu/ml/datasets/iris) as a matrix $X$ of inputs and a vector $\mathbf{y}$ of class labels in the set $\{0, 1, 2\}$ for classes 1, 2, and 3, which are different species of iris flowers.

In [53]:
import numpy as np
import matplotlib.pyplot as plt

# Import iris.data (CSV file)

with open('/home/mdailey/work/PMDS FoML Exercise 1/iris.data', encoding="utf-8") as f:
    read_data = f.read()

X = []
Y_labels = []
for line in read_data.splitlines():
    data = line.split(',')
    if len(data) < 5:
        break
    x = data[:4]
    X.append(x)
    y = data[4]
    Y_labels.append(y)

m = len(Y_labels)
X = np.concatenate((np.ones((m, 1)), np.array(X, np.float32)), 1)
Y_labels = np.array(Y_labels)
y = np.zeros((m, 1))
y[Y_labels == 'Iris-setosa'] = 0
y[Y_labels == 'Iris-versicolor'] = 1
y[Y_labels == 'Iris-virginica'] = 2

K = 3

# Inputs are now in X, and outputs are in y


Here is the hypothesis function outputting all $\phi_k$ predictions for all $m$ values in matrix $X$, a cost
function with gradient calculation, and a gradient descent loop:

In [78]:
theta = np.zeros((K, X.shape[1]))

def hypothesis(X, theta):
    K = theta.shape[0]
    m = X.shape[0]
    a = np.zeros((m, K))
    for k in range(K):
        a[:,k:k+1] = X @ theta[k:k+1,:].T
    e_a = np.exp(a)
    return e_a / e_a.sum(axis=1, keepdims=True)

def onehot(y, K):
    Y = np.zeros((y.shape[0], K))
    for k in range(K):
        Y[y.reshape(-1)==k,k] = 1
    return Y
    
def cost(X, y, theta):
    K = theta.shape[0]
    m = X.shape[0]
    yhat = hypothesis(X, theta)
    # Form the matrix delta(y=k)
    Y = onehot(y, K)
    J = 0
    for k in range(K):
        J = J - np.log(yhat[y.reshape(-1)==k,k]).sum()
    J = J / m
    # Get the gradient matrix for theta
    grad_J = (yhat - Y).T @ X
    return J, grad_J

alpha = 0.001
for epoch in range(100000):
    J, grad_J = cost(X, y, theta)
    if epoch % 1000 == 0:
        print('Epoch %d: cost %f' % (epoch, J))
    theta = theta - alpha * grad_J


Epoch 0: cost 1.098612
Epoch 1000: cost 0.102589
Epoch 2000: cost 0.082405
Epoch 3000: cost 0.073794
Epoch 4000: cost 0.068795
Epoch 5000: cost 0.065431
Epoch 6000: cost 0.062962
Epoch 7000: cost 0.061044
Epoch 8000: cost 0.059493
Epoch 9000: cost 0.058202
Epoch 10000: cost 0.057103
Epoch 11000: cost 0.056151
Epoch 12000: cost 0.055316
Epoch 13000: cost 0.054573
Epoch 14000: cost 0.053907
Epoch 15000: cost 0.053304
Epoch 16000: cost 0.052755
Epoch 17000: cost 0.052253
Epoch 18000: cost 0.051790
Epoch 19000: cost 0.051362
Epoch 20000: cost 0.050965
Epoch 21000: cost 0.050594
Epoch 22000: cost 0.050248
Epoch 23000: cost 0.049923
Epoch 24000: cost 0.049617
Epoch 25000: cost 0.049329
Epoch 26000: cost 0.049057
Epoch 27000: cost 0.048799
Epoch 28000: cost 0.048554
Epoch 29000: cost 0.048322
Epoch 30000: cost 0.048101
Epoch 31000: cost 0.047891
Epoch 32000: cost 0.047690
Epoch 33000: cost 0.047497
Epoch 34000: cost 0.047314
Epoch 35000: cost 0.047138
Epoch 36000: cost 0.046969
Epoch 37000: c

Next we calculate the (training set) accuracy:

In [94]:
preds = hypothesis(X, theta)
pred_classes = np.argmax(preds, axis=1)
print('Training set accuracy: %f' % ((pred_classes==y.reshape(-1)).sum()/m))

Training set accuracy: 0.986667


Finally, let's understand the softmax function in a bit more detail. The softmax refers to the idea that we
would like to find the maximum output of a linear transformation of the inputs, but keep the information
about the relative contribution of the $K$ different outputs.

The softmax function, for a vector of activations $\mathbf{a}$, is defined as
$$\phi_k = \frac{e^{a_k}}{\sum_{j=1}^K e^{a_j}}$$

In [99]:
linear_activations = np.array([ -2, 0.5, -1, 1 ])

def softmax(a):
    return np.exp(a) / np.exp(a).sum()

def hardmax(a):
    phi = np.zeros(len(a))
    phi[a.argmax()] = 1
    return phi

print(softmax(linear_activations))
print(hardmax(linear_activations))

[0.02778834 0.33853132 0.07553655 0.55814379]
[0. 0. 0. 1.]
